In [1]:
import t2_flujo as T2

Matriz de Costos de Transporte entre Nodos:
             10827367167  7564594729   12119644005  475560866    1143298979   \
10827367167     0.000000     0.024501     0.103508     0.083034     0.053149   
7564594729      0.024501     0.000000     0.104909     0.083630     0.068591   
12119644005     0.103508     0.104909     0.000000     0.021280     0.065588   
475560866       0.083034     0.083630     0.021280     0.000000     0.051228   
1143298979      0.053149     0.068591     0.065588     0.051228     0.000000   
3723703649      0.021390     0.035465     0.124746     0.104385     0.070805   
1853718397      0.018246     0.016305     0.091469     0.070343     0.052391   
1784870456      0.093925     0.097952     0.013626     0.017892     0.052661   
267187616       0.063376     0.061044     0.044227     0.023098     0.046573   
1886717444      0.071622     0.073375     0.032053     0.011570     0.041529   
321004778       0.018803     0.027681     0.084943     0.064312     0.040914